In [59]:
import numpy as np
import pandas as pd

In [60]:
playlists_df = pd.read_csv('spotify_playlists.csv', sep=';')
playlists_df = playlists_df.rename(index=str, columns={'trackIds': 'track_ids', 'numTracks': 'num_tracks'})

playlists_df = playlists_df[playlists_df.followers != 'null']
playlists_df['followers'] = [int(i) for i in playlists_df['followers']]

playlists_df['track_ids'] = [i.replace(',null', '').replace('"','').replace('[','').replace(']','').split(',') for i in playlists_df['track_ids']]

playlists_df['num_tracks'] = [int(len(i)) for i in playlists_df['track_ids']]

playlists_df.head()

,id,followers,name,owner,track_ids,num_tracks
0,37i9dQZF1DXcBWIGoYBM5M,18129916,Today's Top Hits,spotify,"[0tBbt8CrmxbjRP0pueQkyU, 2amzBJRBPOGszBem4Fedf...",50
1,37i9dQZF1DX0XUsuxWHRQd,8323892,RapCaviar,spotify,"[4dVpf9jZjcORqGTLUaeYj9, 3ncgNpxLoBQ65ABk4djDy...",51
2,37i9dQZF1DX4dyzvuaRJ0n,4600937,mint,spotify,"[3VQDpxMffTaggOHEeur7Tj, 43V1z6QToNNWNldV4FEG6...",52
3,37i9dQZF1DX4SBhb3fqCJd,3787551,Are & Be,spotify,"[6gU9OKjOE7ghfEd55oRO57, 25wStx3LyTjYmHTd3RDu0...",51
4,37i9dQZF1DXcF6B6QPhFDv,3996091,Rock This,spotify,"[3k7JQg9M5rZJHveFYq0y9J, 5UMD1Iz7yyVK8Q5FLsVt3...",50


In [63]:
tracks_df = pd.read_csv('tracks.csv', sep=';', error_bad_lines=False)
tracks_df.head()

,album_name,album_id,artist_names,artist_ids,available_markets,track_id,track_isrc_id,track_name,track_number,disc_number,explicit,popularity,names,artist_ids.1
0,Lecture I.1.2 The Individual in Oriental Mytho...,5ixfws3clLJYNv7G1WXtH2,"[""Joseph Campbell""]","[""5n9GpZoADUf74D92NYi6CN""]","[""AD"",""AR"",""AT"",""AU"",""BE"",""BG"",""BO"",""BR"",""CA"",...",4V5lOCrpJfNDcYDO6i3S2F,QMHEZ1300019,Individual Identity in the Occident and Orient,2.0,1.0,false,12.0,NaN,NaN
1,Creedence Clearwater Revival - Best Of (Deluxe),1CyJXMyxs8q1X2UdN0ZT4Q,"[""Creedence Clearwater Revival""]","[""3IYUhFvPQItj6xySrBmZkd""]","[""AU"",""CZ"",""EE"",""FI"",""FR"",""GB"",""HK"",""IE"",""LT"",...",3gKk0KYBayVp07RmlXOT6e,USFI87000058,Up Around The Bend,24.0,1.0,false,40.0,NaN,NaN
2,Kayleigh O'connor EP,7lRmQCOro05xMNLjGfA0KZ,"[""Kayleigh O'connor""]","[""0PXv6AATOrRuhWmExZHGRE""]","[""AD"",""AR"",""AT"",""AU"",""BE"",""BG"",""BO"",""BR"",""CA"",...",6rOt8oPfDvBVlNnqSyKnnZ,CAH0R1400006,Hurricane,6.0,1.0,false,31.0,NaN,NaN
3,Ask Me About My New God!,3NDNvu4qTkb6gnh5XJJ1zi,"[""Maria Bamford""]","[""7bF1kZnhlO55twMXqZQIhX""]","[""AD"",""AR"",""AT"",""AU"",""BE"",""BG"",""BO"",""BR"",""CA"",...",632rEUly1PVmwnmRj1ZmeR,US85P1317117,Romantic Expectations of Everyone,17.0,1.0,true,26.0,NaN,NaN
4,The Rise,1ST1X21yXzG5rJtflnBXFT,"[""Futuristic""]","[""5Z5jUyiNvFaqp0EVyLNf0p""]","[""AD"",""AR"",""AT"",""AU"",""BE"",""BG"",""BO"",""BR"",""CA"",...",7JpewU9FPkJyiOi0CErkpI,TCACE1588349,The Greatest,1.0,1.0,true,62.0,NaN,NaN


In [64]:
tracks_df = pd.read_csv('tracks.csv', sep=';', error_bad_lines=False)
tracks_df = tracks_df.drop(['names','artist_ids.1'], axis=1)
tracks_df['explicit'] = [1 if i == 'true' else 0 for i in tracks_df['explicit']]
tracks_df = tracks_df.dropna(axis=0, how='any')
tracks_df = tracks_df[~tracks_df.available_markets.str.contains('0')]
tracks_df = tracks_df[~tracks_df.available_markets.str.contains('1')]
tracks_df = tracks_df.reset_index()

tracks_df['artist_names'] = [i.replace('"','').replace('[','').replace(']','').split(',') for i in tracks_df['artist_names']]
tracks_df['artist_ids'] = [i.replace('"','').replace('[','').replace(']','').split(',') for i in tracks_df['artist_ids']]
tracks_df['available_markets'] = [i.replace('"','').replace('[','').replace(']','').split(',') for i in tracks_df['available_markets']]
tracks_df['num_artists'] = [int(len(i)) for i in tracks_df['artist_ids']]
tracks_df['num_markets'] = [int(len(i)) for i in tracks_df['available_markets']]

audio_features_df = pd.read_csv('audio_features.csv', sep=';')

tracks_df = tracks_df.join(audio_features_df.set_index('id'), on='track_id')
tracks_df.head()

,index,album_name,album_id,artist_names,artist_ids,available_markets,track_id,track_isrc_id,track_name,track_number,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,duration_ms,time_signature
0,0,Lecture I.1.2 The Individual in Oriental Mytho...,5ixfws3clLJYNv7G1WXtH2,[Joseph Campbell],[5n9GpZoADUf74D92NYi6CN],"[AD, AR, AT, AU, BE, BG, BO, BR, CA, CH, CL, C...",4V5lOCrpJfNDcYDO6i3S2F,QMHEZ1300019,Individual Identity in the Occident and Orient,2.0,...,6.0,-12.041,0.0,0.8060,0.7030,0.006230,0.4220,0.2250,265797.0,3.0
1,1,Creedence Clearwater Revival - Best Of (Deluxe),1CyJXMyxs8q1X2UdN0ZT4Q,[Creedence Clearwater Revival],[3IYUhFvPQItj6xySrBmZkd],"[AU, CZ, EE, FI, FR, GB, HK, IE, LT, LV, MY, N...",3gKk0KYBayVp07RmlXOT6e,USFI87000058,Up Around The Bend,24.0,...,2.0,-5.241,1.0,0.0383,0.0228,0.678000,0.1540,0.7360,162853.0,4.0
2,2,Kayleigh O'connor EP,7lRmQCOro05xMNLjGfA0KZ,[Kayleigh O'connor],[0PXv6AATOrRuhWmExZHGRE],"[AD, AR, AT, AU, BE, BG, BO, BR, CA, CH, CL, C...",6rOt8oPfDvBVlNnqSyKnnZ,CAH0R1400006,Hurricane,6.0,...,2.0,-11.428,0.0,0.0322,0.9830,0.000113,0.0938,0.0905,272967.0,4.0
3,3,Ask Me About My New God!,3NDNvu4qTkb6gnh5XJJ1zi,[Maria Bamford],[7bF1kZnhlO55twMXqZQIhX],"[AD, AR, AT, AU, BE, BG, BO, BR, CA, CH, CL, C...",632rEUly1PVmwnmRj1ZmeR,US85P1317117,Romantic Expectations of Everyone,17.0,...,5.0,-10.210,1.0,0.9580,0.8310,0.000001,0.6690,0.5790,115760.0,3.0
4,4,The Rise,1ST1X21yXzG5rJtflnBXFT,[Futuristic],[5Z5jUyiNvFaqp0EVyLNf0p],"[AD, AR, AT, AU, BE, BG, BO, BR, CA, CH, CL, C...",7JpewU9FPkJyiOi0CErkpI,TCACE1588349,The Greatest,1.0,...,9.0,-4.676,1.0,0.3070,0.2100,0.000000,0.2750,0.1970,214903.0,4.0


In [65]:
continuous_var = ['popularity','danceability','energy','loudness','speechiness',
                  'acousticness','instrumentalness','liveness','valence','duration_ms',
                  'num_artists','num_markets']
binary_var = ['explicit','mode']
categorical_var = ['key','time_signature']
all_var = continuous_var + binary_var + categorical_var

def gen_cols(track_list):
    col_values = [[] for i in range(len(all_var))]
    for track in track_list:
        for j in range(len(all_var)):
            val = tracks_df.loc[tracks_df['track_id'] == track, all_var[j]]
            if len(val) > 0:
                col_values[j].append(val.iloc[0])
    return col_values

In [66]:
new_cols = [[] for i in range(len(all_var))]

for playlist in playlists_df['track_ids']:
    values = gen_cols(playlist)
    for i in range(len(values)):
        new_cols[i].append(values[i])

for i in range(len(all_var)):
    playlists_df[all_var[i]] = new_cols[i]

playlists_df.head()

,id,followers,name,owner,track_ids,num_tracks,popularity,danceability,energy,loudness,...,instrumentalness,liveness,valence,duration_ms,num_artists,num_markets,explicit,mode,key,time_signature
0,37i9dQZF1DXcBWIGoYBM5M,18129916,Today's Top Hits,spotify,"[0tBbt8CrmxbjRP0pueQkyU, 2amzBJRBPOGszBem4Fedf...",50,"[88.0, 86.0, 86.0, 47.0, 86.0, 86.0, 91.0, 82....","[0.72, 0.765, 0.751, 0.648, nan, 0.845, 0.643,...","[0.807, 0.688, 0.579, 0.608, nan, 0.709, 0.783...","[-4.59, -5.566, -4.036, -5.16, nan, -4.547, -6...",...,"[0.0, 0.0, 2.28e-05, 0.0, nan, 0.0, 0.0, 0.0, ...","[0.183, 0.0734, 0.133, 0.105, nan, 0.094, 0.08...","[0.305, 0.434, 0.349, 0.488, nan, 0.62, 0.579,...","[197993.0, 229573.0, 199095.0, 190643.0, nan, ...","[2, 1, 2, 1, 4, 2, 2, 3, 1, 2, 2, 2, 2, 3, 1, ...","[3, 3, 62, 1, 3, 62, 62, 62, 62, 3, 62, 59, 60...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, ...","[0.0, 1.0, 1.0, 1.0, nan, 0.0, 1.0, 0.0, nan, ...","[11.0, 4.0, 2.0, 8.0, nan, 10.0, 10.0, 3.0, na...","[4.0, 4.0, 4.0, 4.0, nan, 4.0, 4.0, 4.0, nan, ..."
1,37i9dQZF1DX0XUsuxWHRQd,8323892,RapCaviar,spotify,"[4dVpf9jZjcORqGTLUaeYj9, 3ncgNpxLoBQ65ABk4djDy...",51,"[90.0, 90.0, 74.0, 69.0, 95.0, 72.0, 69.0, 68....","[0.797, 0.95, 0.475, nan, 0.58, 0.605, 0.919, ...","[0.844, 0.533, 0.823, nan, 0.531, 0.784, 0.535...","[-5.482, -7.022, -5.059, nan, -6.631, -7.836, ...",...,"[0.0, 0.0, 0.0, nan, 0.000127, 0.0, 0.0, 0.0, ...","[0.087, 0.114, 0.474, nan, 0.143, 0.71, 0.102,...","[0.52, 0.675, 0.476, nan, 0.141, 0.159, 0.197,...","[173600.0, 124056.0, 220996.0, nan, 218320.0, ...","[1, 1, 1, 1, 2, 2, 5, 1, 2, 2]","[62, 62, 3, 1, 3, 3, 62, 62, 62, 62]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1.0, 1.0, 1.0, nan, 0.0, 1.0, 0.0, 1.0, nan, ...","[11.0, 5.0, 0.0, nan, 5.0, 2.0, 5.0, 9.0, nan,...","[4.0, 4.0, 5.0, nan, 4.0, 4.0, 4.0, 4.0, nan, ..."
2,37i9dQZF1DX4dyzvuaRJ0n,4600937,mint,spotify,"[3VQDpxMffTaggOHEeur7Tj, 43V1z6QToNNWNldV4FEG6...",52,"[64.0, 32.0, 44.0, 70.0, 54.0]","[0.577, 0.828, 0.705, 0.749, 0.735]","[0.507, 0.447, 0.909, 0.755, 0.849]","[-7.027, -9.789, -4.485, -7.5, -4.002]",...,"[0.0, 0.000918, 0.159, 0.00268, 0.00229]","[0.149, 0.11, 0.0877, 0.0835, 0.0703]","[0.245, 0.63, 0.272, 0.313, 0.805]","[237053.0, 187752.0, 342857.0, 210995.0, 19777...","[1, 2, 3, 2, 2]","[62, 1, 3, 62, 62]","[0, 0, 0, 0, 0]","[0.0, 1.0, 1.0, 0.0, 0.0]","[10.0, 6.0, 6.0, 0.0, 3.0]","[4.0, 4.0, 4.0, 4.0, 4.0]"
3,37i9dQZF1DX4SBhb3fqCJd,3787551,Are & Be,spotify,"[6gU9OKjOE7ghfEd55oRO57, 25wStx3LyTjYmHTd3RDu0...",51,"[70.0, 70.0, 81.0, 57.0, 67.0, 66.0, 60.0, 19....","[0.678, 0.612, 0.578, 0.706, nan, 0.668, 0.668...","[0.462, 0.325, 0.574, 0.418, nan, 0.529, 0.519...","[-7.842, -9.497, -6.209, -9.333, nan, -7.804, ...",...,"[0.0, 1.06e-05, 0.0, 2.04e-05, nan, 0.0, 0.0, ...","[0.11, 0.109, 0.083, 0.144, nan, 0.038, 0.0953...","[0.121, 0.0628, 0.301, 0.195, nan, 0.545, 0.39...","[244280.0, 309153.0, 261240.0, 180225.0, nan, ...","[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]","[62, 62, 62, 3, 62, 3, 62, 1, 62, 3, 1]","[0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1]","[1.0, 0.0, 1.0, 1.0, nan, 1.0, 1.0, 0.0, 1.0, ...","[0.0, 6.0, 2.0, 6.0, nan, 9.0, 1.0, 10.0, 2.0,...","[4.0, 4.0, 4.0, 4.0, nan, 4.0, 4.0, 4.0, 4.0, ..."
4,37i9dQZF1DXcF6B6QPhFDv,3996091,Rock This,spotify,"[3k7JQg9M5rZJHveFYq0y9J, 5UMD1Iz7yyVK8Q5FLsVt3...",50,"[60.0, 68.0, 67.0, 61.0, 68.0, 60.0, 64.0, 62.0]","[0.495, 0.615, 0.474, 0.653, 0.669, 0.531, 0.3...","[0.747, 0.74, 0.843, 0.941, 0.777, 0.877, 0.96...","[-5.302, -4.662, -5.909, -2.709, -4.562, -3.82...",...,"[0.000356, 0.00113, 0.104, 0.0788, 5.5e-05, 0....","[0.192, 0.115, 0.362, 0.277, 0.128, 0.0515, 0....","[0.457, 0.219, 0.433, 0.688, 0.523, 0.529, 0.8...","[198080.0, 221707.0, 274282.0, 190448.0, 21470...","[1, 1, 1, 1, 1, 1, 1, 1]","[1, 62, 62, 3, 62, 61, 62, 61]","[0, 0, 0, 0, 0, 0, 0, 0]","[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0]","[0.0, 7.0, 0.0, 1.0, 8.0, 0.0, 1.0, 1.0]","[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]"


In [67]:
playlists_df.to_csv('playlist_data1.csv')

In [75]:
albums_df = pd.read_csv('albums.csv', sep=';')
albums_df = albums_df.dropna(axis=0, how='any')
albums_df = albums_df.rename(index=str, columns={'popularity': 'album_popularity', 'release_date': 'album_release_date'})
albums_df.head()

,album_id,album_type,album_upc_id,genres,label,album_popularity,album_release_date,release_date_precision,available_markets,artist_names,artist_ids,artist_types,track_names,track_ids,copyright_text,copyright_type
0,4uuKYkrV3LYJLg17b0u2Jz,album,8.862965e+11,[],Sounds Of Rain & Thunder Storms,57.0,2016-12-04,day,"[""AD"",""AR"",""AT"",""AU"",""BE"",""BG"",""BO"",""BR"",""CA"",...","[""Sounds Of Rain & Thunder Storms""]","[""1Kua8JL4qKC0Yz5KMoANYU""]","[""artist""]","[""Sounds of Rain"",""Storm Sounds"",""Thunder Soun...","[""5pg3N4FbltCUtPOK5PxDhr"",""572hWrEaDvDALl7pA6b...","[""2016 Sounds Of Rain & Thunder Storms"",""2016 ...","[""C"",""P""]"
1,3aGkjugjzDWaETRbbRafoF,compilation,1.902960e+11,[],Parlophone UK,43.0,1976,year,"[""AD"",""AR"",""AT"",""AU"",""BE"",""BG"",""BO"",""BR"",""CA"",...","[""David Bowie""]","[""0oSGxfWSnnOXhD2fKuz2Gy""]","[""artist""]","[""Space Oddity - 2015 Remastered Version"",""Joh...","[""0waHAu7Lr4vXxP3OWsnwLt"",""6HnFPoxGjTsXe9AZO4r...","[""2016 Jones/Tintoretto Entertainment Co. LLC....","[""C"",""P""]"
2,1vlwV5956aKvDEzuQgMDWX,album,8.800404e+11,[],Onyx Classics,20.0,2013-06-03,day,"[""AD"",""AR"",""AT"",""AU"",""BE"",""BG"",""BO"",""BR"",""CA"",...","[""Pascal Rogé"",""Ami Rogé""]","[""5X5cGjYEmKnuihYXEePQcs"",""4R7GJ1QqIdDhtarC80j...","[""artist"",""artist""]","[""Prélude a l'après-midi d'un faune"",""Fêtes"",""...","[""6RR2GHspdH6Nl6w0pRJIv7"",""5ACNGe4uupESIo3Zg20...","[""(C) 2013 PM Classics Ltd.""]","[""C""]"
3,6Ub7x47sI1SRulOYGvDeHE,album,7.350055e+12,[],Kning Disk/Telegram Studios,13.0,2014-11-10,day,"[""AD"",""AR"",""AT"",""AU"",""BE"",""BG"",""BO"",""BR"",""CA"",...","[""The Hanged Man""]","[""5tZJr2eD6Hzed8goSsvFzN""]","[""artist""]","[""The Island"",""First Time Dreaming"",""Into the ...","[""1HuIyj0e47MoptvZZbERJh"",""5Z8j7HPqz4AKDwX64rr...","[""2014 Kning Disk/Telegram Studios"",""2014 Knin...","[""C"",""P""]"
5,0mwVjzIzef3VieD5B6JtUI,single,8.149080e+11,[],300 Entertainment,74.0,2017-10-12,day,"[""CA"",""NL"",""US""]","[""Cheat Codes"",""Fetty Wap"",""CVBZ""]","[""7DMveApC7UnC2NPfPvlHSU"",""6PXS4YHDkKvl1wkIl4V...","[""artist"",""artist"",""artist""]","[""Feels Great (feat. Fetty Wap & CVBZ)""]","[""5uu28fUesZMl89lf9CLrgN""]","[""2017 Too Easy LLC under exclusive license to...","[""C"",""P""]"


In [76]:
album_var = ['album_type','album_popularity','album_release_date']

def gen_cols(track_list):
    col_values = [[] for i in range(len(album_var))]
    for track in track_list:
        for j in range(len(album_var)):
            album = tracks_df.loc[tracks_df['track_id'] == track, 'album_id']
            if len(album) > 0:
                album = album.iloc[0]
                val = albums_df.loc[albums_df['album_id'] == album, album_var[j]]
                if len(val) > 0:
                    col_values[j].append(val.iloc[0])
    return col_values

new_cols = [[] for i in range(len(album_var))]

for playlist in playlists_df['track_ids']:
    values = gen_cols(playlist)
    for i in range(len(values)):
        new_cols[i].append(values[i])

for i in range(len(album_var)):
    playlists_df[album_var[i]] = new_cols[i]

playlists_df.head()

,id,followers,name,owner,track_ids,num_tracks,popularity,danceability,energy,loudness,...,num_artists,num_markets,explicit,mode,key,time_signature,album_type,release_date,album_popularity,album_release_date
0,37i9dQZF1DXcBWIGoYBM5M,18129916,Today's Top Hits,spotify,"[0tBbt8CrmxbjRP0pueQkyU, 2amzBJRBPOGszBem4Fedf...",50,"[83.0, 81.0, 81.0, 58.0, 81.0, 80.0, 87.0, 77....","[0.72, 0.765, 0.751, 0.648, nan, 0.845, 0.643,...","[0.807, 0.688, 0.579, 0.608, nan, 0.709, 0.783...","[-4.59, -5.566, -4.036, -5.16, nan, -4.547, -6...",...,"[2, 1, 2, 1, 4, 2, 2, 3, 1, 2, 2, 2, 2, 3, 1, ...","[3, 3, 62, 1, 3, 62, 62, 62, 62, 3, 62, 59, 60...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, ...","[0.0, 1.0, 1.0, 1.0, nan, 0.0, 1.0, 0.0, nan, ...","[11.0, 4.0, 2.0, 8.0, nan, 10.0, 10.0, 3.0, na...","[4.0, 4.0, 4.0, 4.0, nan, 4.0, 4.0, 4.0, nan, ...","[single, single, single, album, single, single...","[2017-10-25, 2017-10-20, 2017-11-12, 2017-11-0...","[83.0, 81.0, 81.0, 58.0, 81.0, 80.0, 87.0, 77....","[2017-10-25, 2017-10-20, 2017-11-12, 2017-11-0..."
1,37i9dQZF1DX0XUsuxWHRQd,8323892,RapCaviar,spotify,"[4dVpf9jZjcORqGTLUaeYj9, 3ncgNpxLoBQ65ABk4djDy...",51,"[88.0, 85.0, 81.0, 68.0, 90.0, 81.0, 80.0, 62....","[0.797, 0.95, 0.475, nan, 0.58, 0.605, 0.919, ...","[0.844, 0.533, 0.823, nan, 0.531, 0.784, 0.535...","[-5.482, -7.022, -5.059, nan, -6.631, -7.836, ...",...,"[1, 1, 1, 1, 2, 2, 5, 1, 2, 2]","[62, 62, 3, 1, 3, 3, 62, 62, 62, 62]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1.0, 1.0, 1.0, nan, 0.0, 1.0, 0.0, 1.0, nan, ...","[11.0, 5.0, 0.0, nan, 5.0, 2.0, 5.0, 9.0, nan,...","[4.0, 4.0, 5.0, nan, 4.0, 4.0, 4.0, 4.0, nan, ...","[album, single, album, album, single, album, a...","[2017-08-18, 2017-08-31, 2017-11-17, 2017-07-0...","[88.0, 85.0, 81.0, 68.0, 90.0, 81.0, 80.0, 62....","[2017-08-18, 2017-08-31, 2017-11-17, 2017-07-0..."
2,37i9dQZF1DX4dyzvuaRJ0n,4600937,mint,spotify,"[3VQDpxMffTaggOHEeur7Tj, 43V1z6QToNNWNldV4FEG6...",52,"[60.0, 27.0, 44.0, 65.0, 49.0]","[0.577, 0.828, 0.705, 0.749, 0.735]","[0.507, 0.447, 0.909, 0.755, 0.849]","[-7.027, -9.789, -4.485, -7.5, -4.002]",...,"[1, 2, 3, 2, 2]","[62, 1, 3, 62, 62]","[0, 0, 0, 0, 0]","[0.0, 1.0, 1.0, 0.0, 0.0]","[10.0, 6.0, 6.0, 0.0, 3.0]","[4.0, 4.0, 4.0, 4.0, 4.0]","[single, single, single, single, single]","[2017-11-17, 2016-04-14, 2017-09-29, 2017-06-0...","[60.0, 27.0, 44.0, 65.0, 49.0]","[2017-11-17, 2016-04-14, 2017-09-29, 2017-06-0..."
3,37i9dQZF1DX4SBhb3fqCJd,3787551,Are & Be,spotify,"[6gU9OKjOE7ghfEd55oRO57, 25wStx3LyTjYmHTd3RDu0...",51,"[78.0, 65.0, 89.0, 53.0, 62.0, 61.0, 55.0, 14....","[0.678, 0.612, 0.578, 0.706, nan, 0.668, 0.668...","[0.462, 0.325, 0.574, 0.418, nan, 0.529, 0.519...","[-7.842, -9.497, -6.209, -9.333, nan, -7.804, ...",...,"[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]","[62, 62, 62, 3, 62, 3, 62, 1, 62, 3, 1]","[0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1]","[1.0, 0.0, 1.0, 1.0, nan, 1.0, 1.0, 0.0, 1.0, ...","[0.0, 6.0, 2.0, 6.0, nan, 9.0, 1.0, 10.0, 2.0,...","[4.0, 4.0, 4.0, 4.0, nan, 4.0, 4.0, 4.0, 4.0, ...","[album, single, album, single, single, single,...","[2017-03-24, 2017-06-16, 2016-11-17, 2017-09-2...","[78.0, 65.0, 89.0, 53.0, 62.0, 61.0, 55.0, 14....","[2017-03-24, 2017-06-16, 2016-11-17, 2017-09-2..."
4,37i9dQZF1DXcF6B6QPhFDv,3996091,Rock This,spotify,"[3k7JQg9M5rZJHveFYq0y9J, 5UMD1Iz7yyVK8Q5FLsVt3...",50,"[55.0, 72.0, 62.0, 56.0, 63.0, 55.0, 70.0, 57.0]","[0.495, 0.615, 0.474, 0.653, 0.669, 0.531, 0.3...","[0.747, 0.74, 0.843, 0.941, 0.777, 0.877, 0.96...","[-5.302, -4.662, -5.909, -2.709, -4.562, -3.82...",...,"[1, 1, 1, 1, 1, 1, 1, 1]","[1, 62, 62, 3, 62, 61, 62, 61]","[0, 0, 0, 0, 0, 0, 0, 0]","[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0]","[0.0, 7.0, 0.0, 1.0, 8.0, 0.0, 1.0, 1.0]","[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]","[single, album, single, single, single, single...","[2017-06-06, 2017-09-22, 2017-06-15, 2017-09-0...","[55.0, 72.0, 62.0, 56.0, 63.0, 55.0, 70.0, 57.0]","[2017-06-06, 2017-09-22, 2017-06-15, 2017-09-0..."


In [77]:
playlists_df.to_csv('playlist_data2.csv')

In [71]:
artists_df = pd.read_csv('artists.csv', sep=';')
artists_df = artists_df.dropna(axis=0, how='any')
artists_df = artists_df.rename(index=str, columns={'followers': 'artist_followers','genres': 'artist_genres'})
artists_df.head()

,artist_id,artist_name,artist_popularity,artist_followers,artist_genres
0,6eoJpTIlcuxJNjV5fDzDJH,Live,65.0,213686.0,"[""alternative metal"",""alternative rock"",""pop r..."
1,4lIgSGJ12UAQOpof0PvG4l,Quint Starkie,28.0,433.0,[]
3,7xtlNrmdLZS2sqIkgWewi1,Odessa,48.0,26314.0,[]
5,5InkwkIvy3GThYC2EBLg7s,Spark!,34.0,2688.0,"[""ebm"",""electro-industrial""]"
6,6vbY3hOaCAhC7VjucswgdS,Twista,70.0,272058.0,"[""crunk"",""dance pop"",""dirty south rap"",""gangst..."


In [78]:
artist_var = ['artist_popularity','artist_followers','artist_genres']

def gen_cols(track_list):
    all_artists = []
    for track in track_list:
        for artist_list in tracks_df.loc[tracks_df['track_id'] == track, 'artist_ids']:
            all_artists.extend(artist_list)
            
    col_values = [[] for i in range(len(artist_var))]
    for artist in all_artists:
        for j in range(len(artist_var)):
            val = artists_df.loc[artists_df['artist_id'] == artist, artist_var[j]]
            if len(val) > 0:
                col_values[j].append(val.iloc[0])            

    return col_values

new_cols = [[] for i in range(len(artist_var))]

for playlist in playlists_df['track_ids']:
    values = gen_cols(playlist)
    for i in range(len(values)):
        new_cols[i].append(values[i])

for i in range(len(artist_var)):
    playlists_df[artist_var[i]] = new_cols[i]

playlists_df.head()

,id,followers,name,owner,track_ids,num_tracks,popularity,danceability,energy,loudness,...,mode,key,time_signature,album_type,release_date,album_popularity,album_release_date,artist_popularity,artist_followers,artist_genres
0,37i9dQZF1DXcBWIGoYBM5M,18129916,Today's Top Hits,spotify,"[0tBbt8CrmxbjRP0pueQkyU, 2amzBJRBPOGszBem4Fedf...",50,"[83.0, 81.0, 81.0, 58.0, 81.0, 80.0, 87.0, 77....","[0.72, 0.765, 0.751, 0.648, nan, 0.845, 0.643,...","[0.807, 0.688, 0.579, 0.608, nan, 0.709, 0.783...","[-4.59, -5.566, -4.036, -5.16, nan, -4.547, -6...",...,"[0.0, 1.0, 1.0, 1.0, nan, 0.0, 1.0, 0.0, nan, ...","[11.0, 4.0, 2.0, 8.0, nan, 10.0, 10.0, 3.0, na...","[4.0, 4.0, 4.0, 4.0, nan, 4.0, 4.0, 4.0, nan, ...","[single, single, single, album, single, single...","[2017-10-25, 2017-10-20, 2017-11-12, 2017-11-0...","[83.0, 81.0, 81.0, 58.0, 81.0, 80.0, 87.0, 77....","[2017-10-25, 2017-10-20, 2017-11-12, 2017-11-0...","[93.0, 93.0, 99.0, 94.0, 95.0, 95.0, 89.0, 87....","[6931904.0, 1563225.0, 2060702.0, 1320638.0, 3...","[[""dance pop"",""pop"",""post-teen pop""], [""broste..."
1,37i9dQZF1DX0XUsuxWHRQd,8323892,RapCaviar,spotify,"[4dVpf9jZjcORqGTLUaeYj9, 3ncgNpxLoBQ65ABk4djDy...",51,"[88.0, 85.0, 81.0, 68.0, 90.0, 81.0, 80.0, 62....","[0.797, 0.95, 0.475, nan, 0.58, 0.605, 0.919, ...","[0.844, 0.533, 0.823, nan, 0.531, 0.784, 0.535...","[-5.482, -7.022, -5.059, nan, -6.631, -7.836, ...",...,"[1.0, 1.0, 1.0, nan, 0.0, 1.0, 0.0, 1.0, nan, ...","[11.0, 5.0, 0.0, nan, 5.0, 2.0, 5.0, 9.0, nan,...","[4.0, 4.0, 5.0, nan, 4.0, 4.0, 4.0, 4.0, nan, ...","[album, single, album, album, single, album, a...","[2017-08-18, 2017-08-31, 2017-11-17, 2017-07-0...","[88.0, 85.0, 81.0, 68.0, 90.0, 81.0, 80.0, 62....","[2017-08-18, 2017-08-31, 2017-11-17, 2017-07-0...","[87.0, 92.0, 82.0, 98.0, 99.0, 98.0, 82.0, 92....","[838712.0, 1316895.0, 184502.0, 2321735.0, 206...","[[""dwn trap"",""hip hop"",""indie r&b"",""pop rap"",""..."
2,37i9dQZF1DX4dyzvuaRJ0n,4600937,mint,spotify,"[3VQDpxMffTaggOHEeur7Tj, 43V1z6QToNNWNldV4FEG6...",52,"[60.0, 27.0, 44.0, 65.0, 49.0]","[0.577, 0.828, 0.705, 0.749, 0.735]","[0.507, 0.447, 0.909, 0.755, 0.849]","[-7.027, -9.789, -4.485, -7.5, -4.002]",...,"[0.0, 1.0, 1.0, 0.0, 0.0]","[10.0, 6.0, 6.0, 0.0, 3.0]","[4.0, 4.0, 4.0, 4.0, 4.0]","[single, single, single, single, single]","[2017-11-17, 2016-04-14, 2017-09-29, 2017-06-0...","[60.0, 27.0, 44.0, 65.0, 49.0]","[2017-11-17, 2016-04-14, 2017-09-29, 2017-06-0...","[68.0, 77.0]","[245250.0, 47893.0]","[[""big room"",""deep big room"",""disco house"",""ed..."
3,37i9dQZF1DX4SBhb3fqCJd,3787551,Are & Be,spotify,"[6gU9OKjOE7ghfEd55oRO57, 25wStx3LyTjYmHTd3RDu0...",51,"[78.0, 65.0, 89.0, 53.0, 62.0, 61.0, 55.0, 14....","[0.678, 0.612, 0.578, 0.706, nan, 0.668, 0.668...","[0.462, 0.325, 0.574, 0.418, nan, 0.529, 0.519...","[-7.842, -9.497, -6.209, -9.333, nan, -7.804, ...",...,"[1.0, 0.0, 1.0, 1.0, nan, 1.0, 1.0, 0.0, 1.0, ...","[0.0, 6.0, 2.0, 6.0, nan, 9.0, 1.0, 10.0, 2.0,...","[4.0, 4.0, 4.0, 4.0, nan, 4.0, 4.0, 4.0, 4.0, ...","[album, single, album, single, single, single,...","[2017-03-24, 2017-06-16, 2016-11-17, 2017-09-2...","[78.0, 65.0, 89.0, 53.0, 62.0, 61.0, 55.0, 14....","[2017-03-24, 2017-06-16, 2016-11-17, 2017-09-2...","[85.0, 70.0, 92.0, 57.0, 69.0, 76.0, 78.0, 88....","[2362220.0, 227393.0, 8960319.0, 20341.0, 3616...","[[""dance pop"",""dwn trap"",""indie r&b"",""pop"",""po..."
4,37i9dQZF1DXcF6B6QPhFDv,3996091,Rock This,spotify,"[3k7JQg9M5rZJHveFYq0y9J, 5UMD1Iz7yyVK8Q5FLsVt3...",50,"[55.0, 72.0, 62.0, 56.0, 63.0, 55.0, 70.0, 57.0]","[0.495, 0.615, 0.474, 0.653, 0.669, 0.531, 0.3...","[0.747, 0.74, 0.843, 0.941, 0.777, 0.877, 0.96...","[-5.302, -4.662, -5.909, -2.709, -4.562, -3.82...",...,"[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0]","[0.0, 7.0, 0.0, 1.0, 8.0, 0.0, 1.0, 1.0]","[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]","[single, album, single, single, single, single...","[2017-06-06, 2017-09-22, 2017-06-15, 2017-09-0...","[55.0, 72.0, 62.0, 56.0, 63.0, 55.0, 70.0, 57.0]","[2017-06-06, 

In [79]:
playlists_df.to_csv('playlist_data3.csv')

In [82]:
playlists_df_1 = pd.read_csv('playlist_data1.csv')
playlists_df_3 = pd.read_csv('playlist_data3.csv')
playlists_df_3['popularity'] = playlists_df_1['popularity']
playlists_df_3.head()

,Unnamed: 0,id,followers,name,owner,track_ids,num_tracks,popularity,danceability,energy,...,mode,key,time_signature,album_type,release_date,album_popularity,album_release_date,artist_popularity,artist_followers,artist_genres
0,0,37i9dQZF1DXcBWIGoYBM5M,18129916,Today's Top Hits,spotify,"['0tBbt8CrmxbjRP0pueQkyU', '2amzBJRBPOGszBem4F...",50,"[88.0, 86.0, 86.0, 47.0, 86.0, 86.0, 91.0, 82....","[0.71999999999999997, 0.76500000000000001, 0.7...","[0.80700000000000005, 0.68799999999999994, 0.5...",...,"[0.0, 1.0, 1.0, 1.0, nan, 0.0, 1.0, 0.0, nan, ...","[11.0, 4.0, 2.0, 8.0, nan, 10.0, 10.0, 3.0, na...","[4.0, 4.0, 4.0, 4.0, nan, 4.0, 4.0, 4.0, nan, ...","['single', 'single', 'single', 'album', 'singl...","['2017-10-25', '2017-10-20', '2017-11-12', '20...","[83.0, 81.0, 81.0, 58.0, 81.0, 80.0, 87.0, 77....","['2017-10-25', '2017-10-20', '2017-11-12', '20...","[93.0, 93.0, 99.0, 94.0, 95.0, 95.0, 89.0, 87....","[6931904.0, 1563225.0, 2060702.0, 1320638.0, 3...","['[""dance pop"",""pop"",""post-teen pop""]', '[""bro..."
1,1,37i9dQZF1DX0XUsuxWHRQd,8323892,RapCaviar,spotify,"['4dVpf9jZjcORqGTLUaeYj9', '3ncgNpxLoBQ65ABk4d...",51,"[90.0, 90.0, 74.0, 69.0, 95.0, 72.0, 69.0, 68....","[0.79700000000000004, 0.94999999999999996, 0.4...","[0.84400000000000008, 0.53299999999999992, 0.8...",...,"[1.0, 1.0, 1.0, nan, 0.0, 1.0, 0.0, 1.0, nan, ...","[11.0, 5.0, 0.0, nan, 5.0, 2.0, 5.0, 9.0, nan,...","[4.0, 4.0, 5.0, nan, 4.0, 4.0, 4.0, 4.0, nan, ...","['album', 'single', 'album', 'album', 'single'...","['2017-08-18', '2017-08-31', '2017-11-17', '20...","[88.0, 85.0, 81.0, 68.0, 90.0, 81.0, 80.0, 62....","['2017-08-18', '2017-08-31', '2017-11-17', '20...","[87.0, 92.0, 82.0, 98.0, 99.0, 98.0, 82.0, 92....","[838712.0, 1316895.0, 184502.0, 2321735.0, 206...","['[""dwn trap"",""hip hop"",""indie r&b"",""pop rap"",..."
2,2,37i9dQZF1DX4dyzvuaRJ0n,4600937,mint,spotify,"['3VQDpxMffTaggOHEeur7Tj', '43V1z6QToNNWNldV4F...",52,"[64.0, 32.0, 44.0, 70.0, 54.0]","[0.57700000000000007, 0.82799999999999996, 0.7...","[0.50700000000000001, 0.44700000000000001, 0.9...",...,"[0.0, 1.0, 1.0, 0.0, 0.0]","[10.0, 6.0, 6.0, 0.0, 3.0]","[4.0, 4.0, 4.0, 4.0, 4.0]","['single', 'single', 'single', 'single', 'sing...","['2017-11-17', '2016-04-14', '2017-09-29', '20...","[60.0, 27.0, 44.0, 65.0, 49.0]","['2017-11-17', '2016-04-14', '2017-09-29', '20...","[68.0, 77.0]","[245250.0, 47893.0]","['[""big room"",""deep big room"",""disco house"",""e..."
3,3,37i9dQZF1DX4SBhb3fqCJd,3787551,Are & Be,spotify,"['6gU9OKjOE7ghfEd55oRO57', '25wStx3LyTjYmHTd3R...",51,"[70.0, 70.0, 81.0, 57.0, 67.0, 66.0, 60.0, 19....","[0.67799999999999994, 0.61199999999999999, 0.5...","[0.46200000000000002, 0.32500000000000001, 0.5...",...,"[1.0, 0.0, 1.0, 1.0, nan, 1.0, 1.0, 0.0, 1.0, ...","[0.0, 6.0, 2.0, 6.0, nan, 9.0, 1.0, 10.0, 2.0,...","[4.0, 4.0, 4.0, 4.0, nan, 4.0, 4.0, 4.0, 4.0, ...","['album', 'single', 'album', 'single', 'single...","['2017-03-24', '2017-06-16', '2016-11-17', '20...","[78.0, 65.0, 89.0, 53.0, 62.0, 61.0, 55.0, 14....","['2017-03-24', '2017-06-16', '2016-11-17', '20...","[85.0, 70.0, 92.0, 57.0, 69.0, 76.0, 78.0, 88....","[2362220.0, 227393.0, 8960319.0, 20341.0, 3616...","['[""dance pop"",""dwn trap"",""indie r&b"",""pop"",""p..."
4,4,37i9dQZF1DXcF6B6QPhFDv,3996091,Rock This,spotify,"['3k7JQg9M5rZJHveFYq0y9J', '5UMD1Iz7yyVK8Q5FLs...",50,"[60.0, 68.0, 67.0, 61.0, 68.0, 60.0, 64.0, 62.0]","[0.495, 0.61499999999999999, 0.473999999999999...","[0.747, 0.73999999999999999, 0.842999999999999...",...,"[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0]","[0.0, 7.0, 0.0, 1.0, 8.0, 0.0, 1.0, 1.0]","[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]","['single', 'album', 'single', 'single', 'singl...","['2017-06-06', '2017-09-22', '2017-06-15', '20...","[55.0, 72.0, 62.0, 56.0, 63.0, 55.0, 70.0, 57.0]","['2017-06-06', '2017-09-22', '2017-06-15', '20...","[64.0, 81.0, 79.0, 76.0, 74.0, 78.0, 86.0, 78.0]","[104625.0, 1522366.0, 1113363.0, 675905.0, 245...","['[""alt-indie rock"",""alternative dance"",""alter..."


In [83]:
playlists_df_3.to_csv('playlist_data_final.csv')